![banner_etl](https://github.com/cistelsa/Commerce_Data_Analysis_and_Recommendations/blob/main/5_Sources/Images/banner_etl.gif?raw=true)

<p align="center">

## **Normalización de Tablas - ETL**

</p>

Normalización de tablas para los detalles de los hoteles de Hotelbeds

<mark style="background:#2bfe9c">Script de automatización</mark>

In [1]:
# Usaremos librería Pandas y Json para permitir la lectura de los archivos
import pandas as pd
import json

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 3, Finished, Available)

In [2]:
# Ruta base donde se encuentran los archivos JSON
path_base = "/lakehouse/default/Files/data/original/HotelBeds/"

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 4, Finished, Available)

### Realizamos lectura del dataset

In [3]:
# Hacemos lectura del dataset de detalles de hoteles.
df_hotels_d = pd.read_json(path_base + "hotels_details_dataset.json")

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 5, Finished, Available)

In [4]:
# Revisamos su estructura, tipos de datos
df_hotels_d.info()

StatementMeta(, 67e3e060-70e5-4840-a54e-0021193c2c19, 6, Finished, Available)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39988 entries, 0 to 39987
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   code               39988 non-null  int64 
 1   name               39988 non-null  object
 2   description        39948 non-null  object
 3   country            39988 non-null  object
 4   state              39988 non-null  object
 5   destination        39988 non-null  object
 6   zone               39988 non-null  object
 7   coordinates        39988 non-null  object
 8   category           39988 non-null  object
 9   categoryGroup      39901 non-null  object
 10  chain              33576 non-null  object
 11  accommodationType  39988 non-null  object
 12  boards             38303 non-null  object
 13  segments           29838 non-null  object
 14  address            39988 non-null  object
 15  postalCode         39929 non-null  object
 16  city               39988 non-null  objec

In [4]:
df_hotels_d.head()
pd.options.display.max_columns=0

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 6, Finished, Available)

Preparamos un script para poder determinar que columnas están anidadas en forma de diccionarios, listas o formatos JSON

Requerimos la columna **code** que es la que nos identifica en la otra tabla

La **Lista de Diccionarios** hace un llamado a los detalles de los servicios que tiene el Hotel, estos se pueden manejar en su mayoría como una tabla relacionada, para este es necesario un proceso de ETL y normalizado de tablas.

En su mayoría los datos anidados como **Ninguno** son varibles del hotel los cuales ya contiene el otro dataset

Podemos observar que las columnas que son en forma de **Diccionarios** como: `name`, `description`, `coordinates`, `address`, `city` estas columnas ya se desanidaron en el otro dataset, las restantes son importantes detalles del hotel.

Eliminamos algunas columnas que ya se encuentran en el dataset de `hotels_dataset.csv`, es el caso de `name, description, coordinates, address, city, email, phones, license, web, S2C, ranking, postalCode, coordinates` en otros casos como el de `license` por tener el 99% de campos vacíos se procede a eliminar la columna ya verificando que los datos no son relevantes.

In [5]:
# Eliminar columnas con datos null: 'name', 'description', 'coordinates', 'address', 'city', 'email', 'phones', 'license', 'web', 'S2C', 'ranking', 'postalCode'
df_hotels_d = df_hotels_d.drop(columns=['name', 'description', 'coordinates', 'address', 'city', 'email', 'phones', 'license', 'web', 'S2C', 'ranking', 'postalCode'])

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 7, Finished, Available)

In [6]:
# Eliminación de duplicados por inconsistencias en la extracción
df_hotels_d.drop_duplicates(subset=["code"], inplace=True, ignore_index=True)

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 8, Finished, Available)

In [7]:
df_hotels_d.info()

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 9, Finished, Available)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39963 entries, 0 to 39962
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   code               39963 non-null  int64 
 1   country            39963 non-null  object
 2   state              39963 non-null  object
 3   destination        39963 non-null  object
 4   zone               39963 non-null  object
 5   category           39963 non-null  object
 6   categoryGroup      39876 non-null  object
 7   chain              33551 non-null  object
 8   accommodationType  39963 non-null  object
 9   boards             38278 non-null  object
 10  segments           29813 non-null  object
 11  rooms              39961 non-null  object
 12  facilities         39963 non-null  object
 13  interestPoints     9030 non-null   object
 14  images             38866 non-null  object
 15  wildcards          27430 non-null  object
 16  lastUpdate         39963 non-null  objec

In [8]:
df_hotels_d.head(2)

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 10, Finished, Available)

,code,country,state,destination,zone,category,categoryGroup,chain,accommodationType,boards,segments,rooms,facilities,interestPoints,images,wildcards,lastUpdate,terminals,issues
0,6474,"{'code': 'US', 'isoCode': 'US', 'description':...","{'code': 'IL', 'name': 'ILLINOIS'}","{'code': 'ORD', 'name': {'content': 'Chicago -...","{'zoneCode': 2, 'name': 'Downtown', 'descripti...","{'code': '4EST', 'description': {'content': '4...","{'code': 'GRUPO4', 'description': {'content': ...","{'code': 'HILTO', 'description': {'content': '...","{'code': 'H', 'typeMultiDescription': {'conten...","[{'code': 'BB', 'description': {'content': 'BE...","[{'code': 34, 'description': {'content': 'Busi...","[{'roomCode': 'ROO.RO-1', 'isParentRoom': True...","[{'facilityCode': 30, 'facilityGroupCode': 10,...","[{'facilityCode': 10, 'facilityGroupCode': 100...","[{'type': {'code': 'HAB', 'description': {'con...","[{'roomType': 'ROO.LK', 'roomCode': 'ROO', 'ch...",2023-07-28,NaN,NaN
1,6478,"{'code': 'US', 'isoCode': 'US', 'description':...","{'code': 'CA', 'name': 'CALIFORNIA'}","{'code': 'LAX', 'name': {'content': 'Los Angel...","{'zoneCode': 9, 'name': 'Los Angeles Internati...","{'code': '3EST', 'description': {'content': '3...","{'code': 'GRUPO3', 'description': {'content': ...","{'code': 'MARIO', 'description': {'content': '...","{'code': 'H', 'typeMultiDescription': {'conten...","[{'code': 'BB', 'description': {'content': 'BE...","[{'code': 34, 'description': {'content': 'Busi...","[{'roomCode': 'DBL.2Q-NM', 'isParentRoom': Fal...","[{'facilityCode': 30, 'facilityGroupCode': 10,...",NaN,"[{'type': {'code': 'RES', 'description': {'con...",NaN,2023-07-28,NaN,NaN


Desanidando las columnas que contienen como tipo de dato 'diccionario'

In [9]:
#Se realiza un for para conocer las columans con ripo de dato diccionario
for i in df_hotels_d.columns:
    if type(df_hotels_d.loc[0,i]) == dict:
        print(i)

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 11, Finished, Available)

country
state
destination
zone
category
categoryGroup
chain
accommodationType


Trabajando con la columna 'country', cuyo valor se sustituye por las iniciales del país al que hace referencia la data extraída

In [39]:
# Crear dataframe vacío para desanidar la información de la columna country
df_countries = pd.DataFrame()

# Desanidamos y en una primera columna "country_code" insertamos el contenido de "code"
df_countries['country_code'] = df_hotels_d['country'].apply(pd.Series)['code']
# Desanidamos y en la segunda columna "country_name" insertamos el contenido de "content" que está dentro de "description"
df_countries['country_name'] = df_hotels_d['country'].apply(pd.Series)['description'].apply(pd.Series)['content']
# Eliminamos duplicados
df_countries.drop_duplicates(inplace=True)
# Hacemos lectura del dataframe
df_countries.head()

StatementMeta(, 7f6ce8af-1cc9-4ee7-a97c-73cf77f34659, 41, Finished, Available)

,country_code,country_name
0,US,United States


In [40]:
# Procedemos a guardar el dataset en csv listo para ser normalizado.
df_countries.to_csv("/lakehouse/default/Files/data/launch/HotelBeds/countries.csv", index=False)

StatementMeta(, 7f6ce8af-1cc9-4ee7-a97c-73cf77f34659, 42, Finished, Available)

Trabajando con la columna 'state'

In [41]:
# Crear dataframe vacío para desanidar la información de la columna state
df_states = pd.DataFrame()

# Desanidamos y en una primera columna "state_code" insertamos el contenido de "code"
df_states['state_code'] = df_hotels_d['state'].apply(pd.Series)['code']
# Desanidamos y en la segunda columna "state_name" insertamos el contenido de "name"
df_states['state_name'] = df_hotels_d['state'].apply(pd.Series)['name']
# Eliminamos duplicados
df_states.drop_duplicates(inplace=True)
# Hacemos lectura del dataframe
df_states.head()

StatementMeta(, 7f6ce8af-1cc9-4ee7-a97c-73cf77f34659, 43, Finished, Available)

,state_code,state_name
0,IL,ILLINOIS
1,CA,CALIFORNIA
3,MA,MASSACHUSETTS
4,LA,LOUISIANA
6,PA,PENNSYLVANIA


In [42]:
# Procedemos a guardar el dataset en csv listo para ser normalizado.
df_states.to_csv("/lakehouse/default/Files/data/launch/HotelBeds/states.csv", index=False)

StatementMeta(, 7f6ce8af-1cc9-4ee7-a97c-73cf77f34659, 44, Finished, Available)

Trabajando con la columna 'destination'

In [11]:
# Crear dataframe vacío para desanidar la información de la columna destination
df_destinations = pd.DataFrame()

# Desanidamos y en una primera columna "destination_code" insertamos el contenido de "content" que está dentro de "name"
df_destinations['destination_code'] = df_hotels_d['destination'].apply(pd.Series)['code']
# Desanidamos y en la segunda columna "destination_name" insertamos el contenido de "name"
df_destinations['destination_name'] = df_hotels_d['destination'].apply(pd.Series)['name'].apply(pd.Series)['content']
# Eliminamos duplicados
df_destinations.drop_duplicates(inplace=True)
# Hacemos lectura del dataframe
df_destinations.head()

StatementMeta(, 67e3e060-70e5-4840-a54e-0021193c2c19, 13, Finished, Available)

,destination_code,destination_name
0,ORD,Chicago - IL
1,LAX,Los Angeles - CA
3,BOS,Boston - MA
4,MSY,New Orleans - LA
6,PHL,Philadelphia - PA


In [12]:
# Procedemos a guardar el dataset en csv listo para ser normalizado.
df_destinations.to_csv("/lakehouse/default/Files/data/launch/HotelBeds/destinations.csv", index=False)

StatementMeta(, 67e3e060-70e5-4840-a54e-0021193c2c19, 14, Finished, Available)

Trabajando con 'zone'

In [10]:
# Crear dataframe vacío para desanidar la información de la columna zone
df_zones = pd.DataFrame()

# Desanidamos y en una primera columna "zone_code" insertamos el contenido de "content" que está dentro de "name"
df_zones['zone_code'] = df_hotels_d['zone'].apply(pd.Series)['zoneCode']
# Desanidamos y en la segunda columna "zone_name" insertamos el contenido de "name"
df_zones['zone_name'] = df_hotels_d['zone'].apply(pd.Series)['name']
# Desanidamos y en la tercera columna "zone_description" insertamos el contenido de "content" que está dentro de "description"
df_zones['zone_description'] = df_hotels_d['zone'].apply(pd.Series)['description'].apply(pd.Series)['content']
# Eliminamos duplicados
df_zones.drop_duplicates(inplace=True)
# Hacemos lectura del dataframe
df_zones.head()

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 12, Finished, Available)

,zone_code,zone_name,zone_description
0,2,Downtown,Downtown
1,9,Los Angeles International Airport,Los Angeles International Airport
2,21,Universal Studios / Burbank Area,Universal Studios / Burbank Area
3,2,Back Bay,Back Bay
4,8,Metairie,Metaire


In [11]:
# Procedemos a guardar el dataset en csv listo para ser normalizado.
df_zones.to_csv("/lakehouse/default/Files/data/launch/HotelBeds/zones.csv", index=False)

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 13, Finished, Available)

Trabajando con category

In [12]:
# Crear dataframe vacío para desanidar la información de la columna category
df_categories = pd.DataFrame()

# Desanidamos y en una primera columna "category_code" insertamos el contenido de "code" 
df_categories['category_code'] = df_hotels_d['category'].apply(pd.Series)['code']
# Desanidamos y en la segunda columna "category_name" insertamos el contenido de "content" que está dentro de description
df_categories['category_name'] = df_hotels_d['category'].apply(pd.Series)['description'].apply(pd.Series)['content']
# Eliminamos duplicados
df_categories.drop_duplicates(inplace=True)
# Hacemos lectura del dataframe
df_categories.head()

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 14, Finished, Available)

,category_code,category_name
0,4EST,4 STARS
1,3EST,3 STARS
3,H4_5,4 STARS AND A HALF
7,H3_5,3 STARS AND A HALF
12,5EST,5 STARS


In [13]:
# Procedemos a guardar el dataset en csv listo para ser normalizado.
df_categories.to_csv("/lakehouse/default/Files/data/launch/HotelBeds/categories.csv", index=False)

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 15, Finished, Available)

Trabajando con CategoryGroup

In [15]:
# Crear dataframe vacío para desanidar la información de la columna categoryGroup
df_categoryGroups = pd.DataFrame()

# Desanidamos y en una primera columna "CategoryGroup_code" insertamos el contenido de "code" 
df_categoryGroups['categorygroup_code'] = df_hotels_d['categoryGroup'].apply(pd.Series)['code']
# Desanidamos y en la segunda columna "category_name" insertamos el contenido de "content" que está dentro de "description"
df_categoryGroups['category_name'] = df_hotels_d['categoryGroup'].apply(pd.Series)['description'].apply(pd.Series)['content']
# Eliminamos duplicados
df_categoryGroups.drop_duplicates(inplace=True)
# Hacemos lectura del dataframe
df_categoryGroups.head()

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 17, Finished, Available)

,categorygroup_code,category_name
0,GRUPO4,"Includes 4-star hotels, 4-superior and 4 de lu..."
1,GRUPO3,Includes 3-star hotels and rural hotels.
12,GRUPO5,Includes 5-star hotel and 5 star luxury.
28,GRUPO2,"Includes 2-star hotels, standard, mini hotels..."
564,GRUPO1,Includes 1-star hotels


In [16]:
# Procedemos a guardar el dataset en csv listo para ser normalizado.
df_categoryGroups.to_csv("/lakehouse/default/Files/data/launch/HotelBeds/categorygroups.csv", index=False)

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 18, Finished, Available)

Trabajando con chain

In [17]:
# Crear dataframe vacío para desanidar la información de la columna chain
df_chains = pd.DataFrame()

# Desanidamos y en una primera columna "chain_code" insertamos el contenido de "code" 
df_chains['chain_code'] = df_hotels_d['chain'].apply(pd.Series)['code']
# Desanidamos y en la segunda columna "chain_name" insertamos el contenido de "content" que está dentro de "description"
df_chains['chain_name'] = df_hotels_d['chain'].apply(pd.Series)['description'].apply(pd.Series)['content']
# Eliminamos duplicados
df_chains.drop_duplicates(inplace=True)
# Hacemos lectura del dataframe
df_chains.head()

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 19, Finished, Available)

,chain_code,chain_name
0,HILTO,HILTON WORLDWIDE
1,MARIO,MARRIOTT HOTELS AND RESORTS
4,WYNDH,WYNDHAM HOTEL GROUP
10,INTCO,IHG HOTELS & RESORTS
11,NaN,NaN


In [18]:
# Procedemos a guardar el dataset en csv listo para ser normalizado.
df_chains.to_csv("/lakehouse/default/Files/data/launch/HotelBeds/chains.csv", index=False)

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 20, Finished, Available)

Trabajando con accommodationType

In [20]:
# Crear dataframe vacío para desanidar la información de la columna accommodationType
df_accommodationTypes = pd.DataFrame()

# Desanidamos y en una primera columna "accommodationtype_code" insertamos el contenido de "code" 
df_accommodationTypes['accommodationtype_code'] = df_hotels_d['accommodationType'].apply(pd.Series)['code']
# Desanidamos y en la segunda columna "accommodationtype_name" insertamos el contenido de "typeDescription"
df_accommodationTypes['accommodationtype_name'] = df_hotels_d['accommodationType'].apply(pd.Series)['typeDescription']
# Eliminamos duplicados
df_accommodationTypes.drop_duplicates(inplace=True)
# Hacemos lectura del dataframe
df_accommodationTypes.head()

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 22, Finished, Available)

,accommodationtype_code,accommodationtype_name
0,H,Hotel
9,W,Resort
42,A,Apartment
47,V,Vacation home or villa
120,R,Vacation resort


In [21]:
# Procedemos a guardar el dataset en csv listo para ser normalizado.
df_accommodationTypes.to_csv("/lakehouse/default/Files/data/launch/HotelBeds/accommodationtypes.csv", index=False)

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 23, Finished, Available)

In [22]:
#Se realiza un for para conocer las columans con ripo de dato diccionario
for i in df_hotels_d.columns:
    if type(df_hotels_d.loc[0,i]) == list:
        print(i)

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 24, Finished, Available)

boards
segments
rooms
facilities
interestPoints
images
wildcards


Trabajando con boards

In [46]:
# Crear dataframe vacío para desanidar la información de la columna boards
df_pre_boards = pd.DataFrame()
# Desanidar la columna 'boards' y seleccionar las claves 'code' y 'description'
df_pre_boards['board_code'] = df_hotels_d['boards'].explode().apply(pd.Series)['code']
# Desanidar la columna 'boards' y seleccionar las claves 'code' y 'description'
df_pre_boards['board_name'] = df_hotels_d['boards'].explode().apply(pd.Series)['description'].apply(pd.Series)['content']

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 48, Finished, Available)

In [47]:
# Combinar df_pre_boards con df_hotels usando la columna 'code' como clave
df_boards = df_pre_boards.merge(df_hotels_d[['code']], left_on=df_pre_boards.index, right_index=True)
# Puedes renombrar las columnas según sea necesario
df_boards.rename(columns={'code_x': 'board_code', 'code': 'hotel_id'}, inplace=True)
df_boards = df_boards.drop(columns=["key_0"])
# Eliminamos campos vacíos
df_boards.dropna(subset=['board_code'], inplace=True)


StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 49, Finished, Available)

In [48]:
df_boards.head(10)

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 50, Finished, Available)

,board_code,board_name,hotel_id
0,BB,BED AND BREAKFAST,6474
0,DB,BUFFET BREAKFAST,6474
0,HB,HALF BOARD,6474
0,FB,FULL BOARD,6474
0,RO,ROOM ONLY,6474
0,AI,ALL INCLUSIVE,6474
1,BB,BED AND BREAKFAST,6478
1,RO,ROOM ONLY,6478
2,BB,BED AND BREAKFAST,6480
2,B2,Breakfast for two guests,6480


In [50]:
# Normalizar tabla columna board_code y hotel_id
df_boards_hotel = df_boards.drop(columns=["board_name"])

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 52, Finished, Available)

In [51]:
df_boards_hotel.head(2)

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 53, Finished, Available)

,board_code,hotel_id
0,BB,6474
0,DB,6474


In [52]:
# Procedemos a guardar el dataset en csv listo para ser normalizado.
df_boards_hotel.to_csv("/lakehouse/default/Files/data/launch/HotelBeds/boards_hotel.csv", index=False)

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 54, Finished, Available)

In [53]:
# Normalizar tabla columna board_code y board_name
df_boards = df_boards.drop(columns=["hotel_id"])
# Eliminamos duplicados
df_boards.drop_duplicates(inplace=True)

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 55, Finished, Available)

In [54]:
df_boards.head(10)

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 56, Finished, Available)

,board_code,board_name
0,BB,BED AND BREAKFAST
0,DB,BUFFET BREAKFAST
0,HB,HALF BOARD
0,FB,FULL BOARD
0,RO,ROOM ONLY
0,AI,ALL INCLUSIVE
2,B2,Breakfast for two guests
16,NaN,NaN
17,LB,LIGHT BREAKFAST
17,AB,AMERICAN BREAKFAST


In [55]:
# Procedemos a guardar el dataset en csv listo para ser normalizado.
df_boards.to_csv("/lakehouse/default/Files/data/launch/HotelBeds/boards.csv", index=False)

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 57, Finished, Available)

Desanidar segments

In [56]:
# Crear dataframe vacío para desanidar la información de la columna segments
df_pre_segments = pd.DataFrame()
# Desanidar la columna 'segments' y seleccionar las claves 'code' y 'description'
df_pre_segments['segment_code'] = df_hotels_d['segments'].explode().apply(pd.Series)['code']
# Desanidar la columna 'segments' y seleccionar las claves 'code' y 'description'
df_pre_segments['segment_name'] = df_hotels_d['segments'].explode().apply(pd.Series)['description'].apply(pd.Series)['content']

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 58, Finished, Available)

In [57]:
# Combinar df_pre_segments con df_hotels usando la columna 'code' como clave
df_segments = df_pre_segments.merge(df_hotels_d[['code']], left_on=df_pre_segments.index, right_index=True)
# Puedes renombrar las columnas según sea necesario
df_segments.rename(columns={'code_x': 'segment_code', 'code': 'hotel_id'}, inplace=True)
df_segments = df_segments.drop(columns=["key_0"])

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 59, Finished, Available)

In [65]:
# Eliminamos campos vacíos
df_segments.dropna(subset=['segment_code'], inplace=True)
# Convertimos "segment_code" en tipo entero
df_segments["segment_code"] = df_segments["segment_code"].astype(int)

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 67, Finished, Available)

In [66]:
df_segments.head(10)

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 68, Finished, Available)

,segment_code,segment_name,hotel_id
0,34.0,Business hotels,6474
0,100.0,Hotels with charm,6474
0,102.0,Green Hotel,6474
0,39.0,Hotels with spa,6474
0,45.0,Sport hotels,6474
1,34.0,Business hotels,6478
2,34.0,Business hotels,6480
3,34.0,Business hotels,6483
3,39.0,Hotels with spa,6483
4,34.0,Business hotels,6487


In [68]:
# Normalizar tabla columna segments_code y hotel_id
df_segments_hotel = df_segments.drop(columns=["segment_name"])

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 70, Finished, Available)

In [69]:
df_segments_hotel.head(2)

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 71, Finished, Available)

,segment_code,hotel_id
0,34,6474
0,100,6474


In [70]:
# Procedemos a guardar el dataset en csv listo para ser normalizado.
df_segments_hotel.to_csv("/lakehouse/default/Files/data/launch/HotelBeds/segments_hotel.csv", index=False)

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 72, Finished, Available)

In [71]:
# Normalizar tabla columna segment_code y segment_name
df_segments = df_segments.drop(columns=["hotel_id"])
# Eliminamos duplicados
df_segments.drop_duplicates(inplace=True)

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 73, Finished, Available)

In [73]:
df_segments.head(10)

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 75, Finished, Available)

,segment_code,segment_name
0,34,Business hotels
0,100,Hotels with charm
0,102,Green Hotel
0,39,Hotels with spa
0,45,Sport hotels
7,36,Golf hotels
8,104,Pet friendly Hotels
8,81,Family hotels
10,37,Beach hotels
34,31,Design


In [74]:
# Procedemos a guardar el dataset en csv listo para ser normalizado.
df_segments.to_csv("/lakehouse/default/Files/data/launch/HotelBeds/segments.csv", index=False)

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 76, Finished, Available)

In [88]:
df_rooms = df_hotels_d['rooms'].explode().apply(pd.Series)

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 90, Finished, Available)

In [89]:
df_rooms.head()

StatementMeta(, 5d515e19-f160-4695-b5e4-6dcbeb309774, 91, Finished, Available)

,roomCode,isParentRoom,minPax,maxPax,maxAdults,maxChildren,minAdults,description,type,characteristic,roomFacilities,roomStays,PMSRoomCode,characteristicCode,0
0,ROO.RO-1,True,1.0,2.0,2.0,1.0,1.0,Room RUN OF HOUSE,"{'code': 'ROO', 'description': {'content': 'Ro...","{'code': 'RO-1', 'description': {'content': 'R...","[{'facilityCode': 220, 'facilityGroupCode': 60...","[{'stayType': 'BED', 'order': '1', 'descriptio...",D2,NaN,NaN
0,DBL.GR-KG,False,1.0,4.0,4.0,3.0,1.0,DOUBLE GUEST ROOM KING BED,"{'code': 'DBL', 'description': {'content': 'DO...","{'code': 'GR-KG', 'description': {'content': '...",NaN,NaN,NaN,NaN,NaN
0,DBL.2Q-EJ,True,1.0,2.0,2.0,1.0,1.0,DOUBLE EXECUTIVE TWO QUEEN BEDS,"{'code': 'DBL', 'description': {'content': 'DO...","{'code': '2Q-EJ', 'description': {'content': '...","[{'facilityCode': 220, 'facilityGroupCode': 60...","[{'stayType': 'BED', 'order': '1', 'descriptio...",Q2T,NaN,NaN
0,JSU.KG-1,False,1.0,5.0,5.0,3.0,1.0,JUNIOR SUITE KING SIZE BED,"{'code': 'JSU', 'description': {'content': 'JU...","{'code': 'KG-1', 'description': {'content': 'K...",NaN,NaN,NaN,NaN,NaN
0,ROO.KG,True,1.0,2.0,2.0,1.0,1.0,Room KING SIZE BED,"{'code': 'ROO', 'description': {'content': 'Ro...","{'code': 'KG', 'description': {'content': 'KIN...","[{'facilityCode': 295, 'facilityGroupCode': 60...","[{'stayType': 'BED', 'order': '1', 'descriptio...",K1RRV,NaN,NaN
